In [56]:
import re
import numpy as np
import pickle
import os

In [57]:
!pip install unidecode

In [58]:
from google.colab import drive
drive.mount('/content/drive')

# Đường dẫn tới thư mục bạn muốn chuyển đến
path = '/content/drive/MyDrive/DACN3_Spell_VN/'

# Thực hiện việc chuyển đến thư mục
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
!ls model

model_spell_ngram_4.h5	spell_0.9897.h5  spell_0.99.h5
model_spell_ngram_5.h5	spell_0.9899.h5  spell_ngram_5.h5


In [60]:
import re
import numpy as np
from unidecode import unidecode
# Danh sách các chữ cái tiếng Việt cả viết thường và viết hoa
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
# Danh sách các chữ cái tiếng Anh viết thường
letters2=list("abcdefghijklmnopqrstuvwxyz")

# đánh máy
typo={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}

# địa phương
region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}

# nguyên âm
vowel=list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")
# viết tắt
acronym={"không":"ko"," anh":" a","em":"e","biết":"bít","giờ":"h","gì":"j","muốn":"mún","học":"hok","yêu":"iu",
         "chồng":"ck","vợ":"vk"," ông":" ô","được":"đc","tôi":"t",
         "Không":"Ko"," Anh":" A","Em":"E","Biết":"Bít","Giờ":"H","Gì":"J","Muốn":"Mún","Học":"Hok","Yêu":"Iu",
         "Chồng":"Ck","Vợ":"Vk"," Ông":" Ô","Được":"Đc","Tôi":"T",}

# teencode
teen={"ch":"ck","ph":"f","th":"tk","nh":"nk",
      "Ch":"Ck","Ph":"F","Th":"Tk","Nh":"Nk"}

In [61]:
# function for adding mistake( noise)
# Hàm thêm lỗi (nhiễu) vào câu

# Hàm teen_code:
# Nếu một từ nằm trong acronym thì có sắc xuất là pivot*0.5*(1-0.05) bị thây thế bằng từ tương ứng trong acronym
# Nếu một từ nằm trong teen thì có sắc xuất là pivot*0.05 bị thây thế bằng từ tương ứng trong acronym
def teen_code(sentence,pivot):
    random = np.random.uniform(0,1,1)[0]# Tạo một số ngẫu nhiên trong khoảng từ 0 đến 1
    new_sentence=str(sentence)# Sao chép câu đầu vào
    if random>pivot:# Nếu số ngẫu nhiên lớn hơn pivot
        for word in acronym.keys():# Duyệt qua các từ khóa trong từ điển viết tắt
            if re.search(word, new_sentence): # Nếu tìm thấy từ khóa trong câu
                random2 = np.random.uniform(0,1,1)[0]# Tạo một số ngẫu nhiên khác
                if random2 <0.5:# Nếu số ngẫu nhiên nhỏ hơn 0.5
                    new_sentence=new_sentence.replace(word,acronym[word])# Thay thế từ khóa bằng từ viết tắt
        for word in teen.keys():# Duyệt qua các từ khóa trong từ điển teen code
            if re.search(word, new_sentence):# Nếu tìm thấy từ khóa trong câu
                random3 = np.random.uniform(0,1,1)[0]# Tạo một số ngẫu nhiên khác
                if random3 <0.05: # Nếu số ngẫu nhiên nhỏ hơn 0.05
                    new_sentence=new_sentence.replace(word,teen[word])# Thay thế từ khóa bằng từ teen code
        return new_sentence# Trả về câu đã được chỉnh sửa
    else:
        return sentence# Trả về câu gốc nếu số ngẫu nhiên không lớn hơn pivot

# Hàm thêm nhiễu vào câu

# Bước 1
# Nếu một từ nằm trong acronym thì có sắc xuất là 0.5*0.5*(1-0.05) bị thây thế bằng từ tương ứng trong acronym
# Nếu một từ nằm trong teen thì có sắc xuất là 0.5*0.05 bị thây thế bằng từ tương ứng trong acronym

# Bước 2
# Trường hợp 1: letters được giữ nguyên có sắc xuất: pivot1
# ...
def add_noise(sentence, pivot1,pivot2):
    sentence=teen_code(sentence,0.5)# Thêm nhiễu dạng teen code vào câu với pivot cố định là 0.5
    noisy_sentence = ""# Chuỗi để lưu câu bị thêm nhiễu
    i = 0 # Chỉ số bắt đầu
    while i < len(sentence):# Duyệt qua từng ký tự trong câu
        if sentence[i] not in letters:# Nếu ký tự không thuộc tập ký tự cho phép
            noisy_sentence+=sentence[i]# Giữ nguyên ký tự
        else:
            random = np.random.uniform(0,1,1)[0]# Tạo một số ngẫu nhiên trong khoảng từ 0 đến 1
            if random < pivot1: # Nếu số ngẫu nhiên nhỏ hơn pivot1
                noisy_sentence+=(sentence[i])# Giữ nguyên ký tự
            elif random<pivot2: # Nếu số ngẫu nhiên lớn hơn pivot1 nhưng nhỏ hơn pivot2
                if sentence[i] in typo.keys() and sentence[i] in region.keys(): # Nếu ký tự có trong cả hai từ điển typo và region
                    random2=np.random.uniform(0,1,1)[0] # Tạo một số ngẫu nhiên khác
                    if random2<=0.4:# Nếu số ngẫu nhiên nhỏ hơn hoặc bằng 0.4
                        noisy_sentence+=typo[sentence[i]]# Thay thế ký tự bằng ký tự trong từ điển typo
                    elif random2<0.8: # Nếu số ngẫu nhiên nhỏ hơn 0.8
                        noisy_sentence+=region[sentence[i]]# Thay thế ký tự bằng ký tự trong từ điển region
                    elif random2<0.95 :# Nếu số ngẫu nhiên nhỏ hơn 0.95
                        noisy_sentence+=unidecode(sentence[i])# Thay thế ký tự bằng phiên bản không dấu
                    else:
                        noisy_sentence+=sentence[i]  # Giữ nguyên ký tự
                elif sentence[i] in typo.keys():# Nếu ký tự chỉ có trong từ điển typo
                    random3=np.random.uniform(0,1,1)[0] # Tạo một số ngẫu nhiên khác
                    if random3<=0.6:# Nếu số ngẫu nhiên nhỏ hơn hoặc bằng 0.6
                        noisy_sentence+=typo[sentence[i]]# Thay thế ký tự bằng ký tự trong từ điển typo
                    elif random3<0.9 :# Nếu số ngẫu nhiên nhỏ hơn 0.9
                        noisy_sentence+=unidecode(sentence[i])# Thay thế ký tự bằng phiên bản không dấu
                    else:
                        noisy_sentence+=sentence[i] # Giữ nguyên ký tự
                elif sentence[i] in region.keys(): # Nếu ký tự chỉ có trong từ điển region
                    random4=np.random.uniform(0,1,1)[0] # Tạo một số ngẫu nhiên khác
                    if random4<=0.6:# Nếu số ngẫu nhiên nhỏ hơn hoặc bằng 0.6
                        noisy_sentence+=region[sentence[i]] # Thay thế ký tự bằng ký tự trong từ điển region
                    elif random4<0.85 : # Nếu số ngẫu nhiên nhỏ hơn 0.85
                        noisy_sentence+=unidecode(sentence[i]) # Thay thế ký tự bằng phiên bản không dấu
                    else:
                        noisy_sentence+=sentence[i]# Giữ nguyên ký tự
                elif i<len(sentence)-1 :# Nếu ký tự không nằm ở cuối câu
                    # Nếu ký tự hiện tại thuộc từ điển region2, nằm ở đầu câu hoặc trước nó không phải là ký tự chữ cái, và ký tự tiếp theo là nguyên âm
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random5=np.random.uniform(0,1,1)[0]# Tạo một số ngẫu nhiên khác
                        if random5<=0.9:# Nếu số ngẫu nhiên nhỏ hơn hoặc bằng 0.9
                            noisy_sentence+=region2[sentence[i]]# Thay thế ký tự bằng ký tự trong từ điển region2
                        else:
                            noisy_sentence+=sentence[i]# Giữ nguyên ký tự
                    else:
                        noisy_sentence+=sentence[i]# Giữ nguyên ký tự

            else:
                new_random = np.random.uniform(0,1,1)[0]# Tạo một số ngẫu nhiên khác
                if new_random <=0.33: # Nếu số ngẫu nhiên nhỏ hơn hoặc bằng 0.33
                    if i == (len(sentence) - 1):
                        continue# Bỏ qua nếu ký tự ở cuối câu
                    else:
                        noisy_sentence+=(sentence[i+1]) # Hoán đổi ký tự hiện tại với ký tự tiếp theo
                        noisy_sentence+=(sentence[i])
                        i += 1 # Tăng chỉ số i lên 1 để bỏ qua ký tự đã được hoán đổi
                elif new_random <= 0.66: # Nếu số ngẫu nhiên nhỏ hơn hoặc bằng 0.66
                    random_letter = np.random.choice(letters2, 1)[0]# Chọn ngẫu nhiên một ký tự từ danh sách letters2
                    noisy_sentence+=random_letter# Thêm ký tự ngẫu nhiên vào câu
                else:
                    pass# Không làm gì cả

        i += 1 # Tăng chỉ số i lên 1 để duyệt ký tự tiếp theo
    return noisy_sentence# Trả về câu đã bị thêm nhiễu


In [62]:
def extract_phrases(text):
    pattern = r'\w[\w ]*\w|\s\W+|\W+|\n'
    return re.findall(pattern, text)

def check_text(regex, text):
    pattern = re.compile(regex)
    if pattern.fullmatch(text):
        return True
    else:
        return False

def add_noise_2(text):
  ep = extract_phrases(text)
  rs_ = ""
  for value in ep:
    regex = r'\w[\w ]+'
    if check_text(regex, value):
      rs_ = rs_ + add_noise(value,0.94,0.985)
    else:
      rs_ = rs_ + value
  return rs_

In [66]:
import pickle
# Tải dữ liệu từ file 'data/VNTC_data.pkl' bằng cách sử dụng pickle và lưu vào biến data
data = pickle.load(open('data/VNTC_data_test.pkl', 'rb'))

# In ra số lượng phần tử trong dữ liệu (len(data))
# Và số lượng phần tử duy nhất trong dữ liệu (len(set(i for i in data)))
print(len(data),len(set(i for i in data)))

14990 14845


In [70]:
import re
import numpy as np
import pickle
from tqdm import tqdm

In [71]:
# Khai báo chuỗi ký tự Latin và các ký tự đặc biệt được phép sử dụng
UNalphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'
# Tạo một danh sách rỗng để lưu trữ dữ liệu huấn luyện sau khi xử lý
test_data=[]
# Duyệt qua từng phần tử trong biến 'data'
for i in tqdm(data, desc="Processing data"):
  # Thay thế ký tự xuống dòng '\n' bằng dấu chấm '.'
  i=i.replace("\n",".")
  # Tách đoạn văn thành các câu dựa trên dấu chấm '.'
  sentences=i.split(".")
  # Duyệt qua từng câu trong danh sách các câu
  for j in sentences:
      # Nếu câu có số từ lớn hơn 2 và khớp với mẫu ký tự được định nghĩa trong 'alphabet'
      if len(j.split()) > 2 and re.match(UNalphabet, j.lower()):
          # Thêm câu vào danh sách 'training_data'
          test_data.append([j, add_noise_2(j)])

# In ra số lượng câu trong dữ liệu huấn luyện sau khi xử lý
print(len(test_data))

Processing data: 100%|██████████| 14990/14990 [03:06<00:00, 80.52it/s]

274155


In [72]:
del data

In [73]:
test_data

[[' Mạo hiểm rừng Đa Mi Cuộc hành quân khám phá thác sương mù Ẩn mình trong cánh rừng Đa Mi (Hàm Thuận Bắc, Bình Thuận) bạt ngàn là hồ thủy điện Đa Mi đẹp như một nàng công chúa, ngọn thác sương mù hùng vĩ, cao ngất, uốn lượn như con rồng bạc khổng lồ',
  ' Mạo ihểm rừng Đa Mi Cuộc hành quân khá mphá thác sương mù Ẩn mình trong cánh rừng Đa Mi (Hàm Thuận Bắc, Bình Thuận) bạt ngàn là hồ thủy điện Đa Mi đẹp như một nàng công chúa, ngọn thác sương mù hùng vĩ, cao ngất, uốn qượn như con rồng bạc khổng lồ'],
 ['Xuyên rừng vào thác sương mù ', 'Xuyên rừng vào thác sương mù '],
 ['Trời đã về chiều khi chúng tôi đến được hồ thủy điện Đa Mi rộng 625ha với rất nhiều hòn đảo nhỏ đẹp như bức tranh thủy mạc dần hiện ra trong ánh hoàng hôn huyền ảo',
  'Trời đã về chiều khi chúng tôi đến đ hồ thủy ddiện Đa Mi rộng 625ha vớq rất nhiều hòn đảo nhỏ đẹp hnư bức tranh thủy mạc dần hiện ra trong asnh hoànw hôn huyền ảo'],
 [' Đến thủy điện nhưng nơi hạ trại lại không có điện',
  ' Đến thủy điện nhưng nowi

In [75]:
import pickle
# Lưu dữ liệu vào file
with open('data/my_data_test.pkl', 'wb') as file:
    pickle.dump(test_data, file)

print("Data saved successfully!")

Data saved successfully!
